In [47]:
from awips.dataaccess import DataAccessLayer
import matplotlib.tri as mtri
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from math import exp, log

import numpy as np
from metpy.calc import get_wind_components, lcl, dry_lapse, parcel_profile, dewpoint
from shapely.geometry import Point
from metpy.plots import SkewT, Hodograph
from metpy.units import units, concatenate
import warnings

from metpy.calc import thermo, vapor_pressure

try:
    from metpy.calc import wind_speed, wind_direction, thermo, vapor_pressure
except:
    def wind_speed(u,v):
        "returns the speed of the wind"
    
        return np.array([(np.linalg.norm( (i,j))**2) for i,j in zip(u,v)])

    def wind_direction(u,v):
        "returns the angle of the wind"
        phi = 180 + 180/np.pi * np.arctan2(𝑣, 𝑢)

        return phi

In [2]:
warnings.filterwarnings("ignore",category =RuntimeWarning)
DataAccessLayer.changeEDEXHost("edex-cloud.unidata.ucar.edu")
request = DataAccessLayer.newDataRequest("modelsounding")
forecastModel = "GFS"
request.addIdentifier("reportType", forecastModel)
request.setParameters("pressure","temperature","specHum","uComp","vComp","omega", "cldCvr")


#Available Locations
locations = DataAccessLayer.getAvailableLocationNames(request)
locations.sort()
list(locations)

['',
 '1V4',
 '3J2',
 '4BL',
 '4BQ',
 '4HV',
 '4OM',
 '5AF',
 '5AG',
 '5SZ',
 '6RO',
 '8V7',
 '9B6',
 'A#2',
 'A#3',
 'A#4',
 'A#5',
 'A#6',
 'A#7',
 'A#8',
 'A#9',
 'A#A',
 'A#B',
 'ABL',
 'ADM',
 'AFA',
 'AGR',
 'AHN',
 'AIA',
 'AIH',
 'AJO',
 'ANJ',
 'APX',
 'AQQ',
 'ATH',
 'ATL1',
 'ATL2',
 'ATL3',
 'ATL4',
 'ATLH',
 'AWH',
 'AWR',
 'B#1',
 'B#2',
 'B#3',
 'B#4',
 'B#5',
 'B#6',
 'B#7',
 'B#8',
 'B#9',
 'B#A',
 'B#B',
 'B#C',
 'B#D',
 'B#E',
 'B#F',
 'B#G',
 'B#H',
 'B#J',
 'B#K',
 'B#L',
 'B#M',
 'B#N',
 'B#O',
 'B#P',
 'B#Q',
 'B#S',
 'BAB',
 'BDG',
 'BDP',
 'BFL',
 'BGTL',
 'BH1',
 'BH2',
 'BH3',
 'BH4',
 'BH5',
 'BHK',
 'BID',
 'BIR',
 'BLS',
 'BLU',
 'BMX',
 'BNA',
 'BOD',
 'BRA',
 'BTL',
 'BVR',
 'C01',
 'C02',
 'C03',
 'C04',
 'C06',
 'C07',
 'C08',
 'C09',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21',
 'C22',
 'C23',
 'C24',
 'C25',
 'C27',
 'C28',
 'C30',
 'C31',
 'C32',
 'C33',
 'C34',
 'C35',
 'C36',
 'C7H',
 'CAI',
 'CAN',
 'CBE',
 'CBN

In [20]:
request.setLocationNames("BRA")
cycles = DataAccessLayer.getAvailableTimes(request, True)
times = DataAccessLayer.getAvailableTimes(request)
try:
    fcstRun = DataAccessLayer.getForecastRun(cycles[-1], times)
    list(fcstRun)
    response = DataAccessLayer.getGeometryData(request,[fcstRun[0]])
except:
    print('No times available')
    

In [21]:
tmp,prs,sh = np.array([]),np.array([]),np.array([])
uc,vc,om,cld = np.array([]),np.array([]),np.array([]),np.array([])
for ob in response:
    tmp = np.append(tmp,ob.getNumber("temperature"))
    prs = np.append(prs,ob.getNumber("pressure"))
    sh = np.append(sh,ob.getNumber("specHum"))
    uc = np.append(uc,ob.getNumber("uComp"))
    vc = np.append(vc,ob.getNumber("vComp"))
    om = np.append(om,ob.getNumber("omega"))
    cld = np.append(cld,ob.getNumber("cldCvr"))
print("parms = " + str(ob.getParameters()))
print("site = " + str(ob.getLocationName()))
print("geom = " + str(ob.getGeometry()))
print("datetime = " + str(ob.getDataTime()))
print("reftime = " + str(ob.getDataTime().getRefTime()))
print("fcstHour = " + str(ob.getDataTime().getFcstTime()))
print("period = " + str(ob.getDataTime().getValidPeriod()))

parms = ['temperature', 'pressure', 'vComp', 'uComp', 'cldCvr', 'specHum', 'omega']
site = BRA
geom = POINT (-84.05000305175781 34.79999923706055)
datetime = 2019-07-15 18:00:00
reftime = Jul 15 19 18:00:00 GMT
fcstHour = 0
period = (Jul 15 19 18:00:00 , Jul 15 19 18:00:00 )


# Calculating Dewpoint from Specific Humidity
Because the modelsounding plugin does not return dewpoint values, we must calculate the profile ourselves. Here
are three examples of dewpoint calculated from specific humidity, including a manual calculation following NCEP
AWIPS/NSHARP.
1) MetPy calculated mixing ratio and vapor pressure

In [57]:
type(units.knots)

pint.unit.build_unit_class.<locals>.Unit

In [64]:
t = (tmp-273.15) * units.degC
p = prs/100 * units.mbar
u,v = uc*1.94384,vc*1.94384 # m/s to knots

spd = wind_speed(u, v) * units.knots


dir = wind_direction(u, v) * units.deg

In [65]:
rmix = (sh/(1-sh)) *1000 * units('g/kg')
e = vapor_pressure(p, rmix)
td = dewpoint(e)

In [66]:
# metpy calculated assuming spec. humidity = mixing ratio

td2 = dewpoint(vapor_pressure(p, sh))

In [67]:
# new arrays
ntmp = tmp
# where p=pressure(pa), T=temp(C), T0=reference temp(273.16)
rh = 0.263*prs*sh / (np.exp(17.67*ntmp/(ntmp+273.15-29.65)))
vaps = 6.112 * np.exp((17.67 * ntmp) / (ntmp + 243.5))
vapr = rh * vaps / 100
dwpc = np.array(243.5 * (np.log(6.112) - np.log(vapr)) / (np.log(vapr) - np.log(6.112) - 17.67)) * units.degC

In [68]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [91]:
plt.rcParams['figure.figsize'] = (12, 14)
# Create a skewT plot
skew = SkewT()
# Plot the data
skew.plot(p, t, 'r', linewidth=2)
skew.plot(p, td, 'b', linewidth=2)
skew.plot(p, td2, 'y')
skew.plot(p, dwpc, 'g', linewidth=2)
skew.plot_barbs(p, u, v)
skew.ax.set_ylim(950, 10)
skew.ax.set_xlim(-80, 40)
plt.title( forecastModel + " " \
+ ob.getLocationName() \
+ "("+ str(ob.getGeometry()) + ")" \
+ ", " + str(ob.getDataTime()) )

# An example of a slanted line at constant T -- in this case the 0 isotherm
l = skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)
# Draw hodograph
ax_hod = inset_axes(skew.ax, '50%', '50%', loc=2)
h = Hodograph(ax_hod, component_range=wind_speed(u, v).max())
h.add_grid(increment=100,)
h.plot_colormapped(u, v, spd)
# Show the plot
plt.show()